Initializing and reading Spark session with MongiDB connector

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum, count, hour, date_format

# Initialize Spark session with MongoDB connector
try:
    spark = SparkSession.builder \
        .appName("DaroogheBatchProcessing") \
        .config("spark.mongodb.input.uri", "mongodb://localhost:27017/darooghe.transactions") \
        .config("spark.mongodb.output.uri", "mongodb://localhost:27017/darooghe.analytics") \
        .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
        .getOrCreate()
    print("Spark session initialized successfully")
except Exception as e:
    print(f"Failed to initialize Spark session: {e}")
    raise

# Read transactions from MongoDB (populated by kafka_consumer.py)
df = spark.read.format("mongo").option("collection", "transactions").load()

Spark session initialized successfully


Comission Analysis Batch Job

In [2]:
from pyspark.sql.functions import col, avg, sum, max as spark_max

# Commission efficiency report
commission_report = df.groupBy("merchant_category") \
    .agg(
        sum("commission_amount").alias("total_commissions"),
        avg("commission_amount").alias("avg_commission"),
        (sum("commission_amount") / sum("amount")).alias("commission_to_transaction_ratio")
    )

# Save to MongoDB
commission_report.write \
    .format("mongo") \
    .option("collection", "commission_reports") \
    .mode("overwrite") \
    .save()

# Simulate optimal commission structures
commission_simulation = df.groupBy("merchant_category", "commission_type") \
    .agg(
        sum("commission_amount").alias("total_commissions"),
        count("transaction_id").alias("transaction_count")
    ) \
    .withColumn("profitability_score", col("total_commissions") / col("transaction_count"))

# Identify optimal commission type per category
optimal_commissions = commission_simulation.alias("sim") \
    .groupBy("merchant_category") \
    .agg(spark_max("profitability_score").alias("profitability_score")) \
    .join(
        commission_simulation.alias("orig"),
        ["merchant_category", "profitability_score"],
        "inner"
    ) \
    .select(
        col("orig.merchant_category"),
        col("orig.commission_type"),
        col("orig.profitability_score")
    )

# Save to MongoDB
optimal_commissions.write \
    .format("mongo") \
    .option("collection", "optimal_commissions") \
    .mode("overwrite") \
    .save()

Transaction Pattern Analysis

In [3]:
# Import necessary libraries
from pyspark.sql.functions import col, hour, when, count, sum, avg, date_format, dayofweek
from pyspark.sql.types import TimestampType

# Ensure timestamp is in correct format
df = df.withColumn("timestamp_dt", col("timestamp").cast(TimestampType()))

# I. Temporal patterns: Transactions by hour and day of week
try:
    temporal_patterns_hour = df.groupBy(hour("timestamp_dt").alias("hour")) \
        .agg(count("transaction_id").alias("transaction_count")) \
        .orderBy("hour")
    
    temporal_patterns_day = df.groupBy(dayofweek("timestamp_dt").alias("day_of_week")) \
        .agg(count("transaction_id").alias("transaction_count")) \
        .orderBy("day_of_week")
    
    # Save to MongoDB
    temporal_patterns_hour.write \
        .format("mongo") \
        .option("uri", "mongodb://localhost:27017/darooghe.temporal_patterns_hour") \
        .mode("overwrite") \
        .save()
    temporal_patterns_day.write \
        .format("mongo") \
        .option("uri", "mongodb://localhost:27017/darooghe.temporal_patterns_day") \
        .mode("overwrite") \
        .save()
    print("Temporal patterns saved to MongoDB")
except Exception as e:
    print(f"Failed to save temporal patterns: {e}")
    raise

# II. Peak transaction times: Top 5 hours
try:
    peak_times = temporal_patterns_hour.orderBy(col("transaction_count").desc()).limit(5)
    peak_times.write \
        .format("mongo") \
        .option("uri", "mongodb://localhost:27017/darooghe.peak_times") \
        .mode("overwrite") \
        .save()
    print("Peak times saved to MongoDB")
except Exception as e:
    print(f"Failed to save peak times: {e}")
    raise

# III. Customer segmentation by spending frequency and total spent
try:
    customer_segments = df.groupBy("customer_id") \
        .agg(
            count("transaction_id").alias("transaction_count"),
            sum("amount").alias("total_spent")
        ) \
        .withColumn("segment",
            when(col("transaction_count") > 20, "High Frequency")
            .when(col("transaction_count") > 5, "Medium Frequency")
            .otherwise("Low Frequency")
        )
    
    customer_segments.write \
        .format("mongo") \
        .option("uri", "mongodb://localhost:27017/darooghe.customer_segments") \
        .mode("overwrite") \
        .save()
    print("Customer segments saved to MongoDB")
except Exception as e:
    print(f"Failed to save customer segments: {e}")
    raise

# IV. Transaction behavior by merchant category
try:
    category_behavior = df.groupBy("merchant_category") \
        .agg(
            avg("amount").alias("avg_transaction_amount"),
            count("transaction_id").alias("transaction_count")
        )
    
    category_behavior.write \
        .format("mongo") \
        .option("uri", "mongodb://localhost:27017/darooghe.category_behavior") \
        .mode("overwrite") \
        .save()
    print("Category behavior saved to MongoDB")
except Exception as e:
    print(f"Failed to save category behavior: {e}")
    raise

# V. Time-of-day segmentation (morning, afternoon, evening, night)
try:
    time_of_day_df = df.withColumn("time_of_day",
        when((hour("timestamp_dt") >= 6) & (hour("timestamp_dt") < 12), "Morning")
        .when((hour("timestamp_dt") >= 12) & (hour("timestamp_dt") < 18), "Afternoon")
        .when((hour("timestamp_dt") >= 18) & (hour("timestamp_dt") < 24), "Evening")
        .otherwise("Night")
    )
    
    time_of_day_patterns = time_of_day_df.groupBy("time_of_day") \
        .agg(count("transaction_id").alias("transaction_count")) \
        .orderBy(col("transaction_count").desc())
    
    time_of_day_patterns.write \
        .format("mongo") \
        .option("uri", "mongodb://localhost:27017/darooghe.time_of_day_patterns") \
        .mode("overwrite") \
        .save()
    print("Time-of-day patterns saved to MongoDB")
except Exception as e:
    print(f"Failed to save time-of-day patterns: {e}")
    raise

# VI. Spending trends over time (by month)
try:
    spending_trends = df.groupBy(date_format("timestamp_dt", "yyyy-MM").alias("year_month")) \
        .agg(sum("amount").alias("total_spending")) \
        .orderBy("year_month")
    
    spending_trends.write \
        .format("mongo") \
        .option("uri", "mongodb://localhost:27017/darooghe.spending_trends") \
        .mode("overwrite") \
        .save()
    print("Spending trends saved to MongoDB")
except Exception as e:
    print(f"Failed to save spending trends: {e}")
    raise

# Display results for verification
print("Temporal Patterns (Hour):")
temporal_patterns_hour.show(truncate=False)
print("Temporal Patterns (Day of Week):")
temporal_patterns_day.show(truncate=False)
print("Peak Times:")
peak_times.show(truncate=False)
print("Customer Segments:")
customer_segments.show(5, truncate=False)
print("Category Behavior:")
category_behavior.show(truncate=False)
print("Time-of-Day Patterns:")
time_of_day_patterns.show(truncate=False)
print("Spending Trends:")
spending_trends.show(truncate=False)

Temporal patterns saved to MongoDB
Peak times saved to MongoDB
Customer segments saved to MongoDB
Category behavior saved to MongoDB
Time-of-day patterns saved to MongoDB
Spending trends saved to MongoDB
Temporal Patterns (Hour):
+----+-----------------+
|hour|transaction_count|
+----+-----------------+
|0   |877              |
|1   |812              |
|2   |806              |
|3   |832              |
|4   |860              |
|5   |843              |
|6   |813              |
|7   |834              |
|8   |839              |
|9   |867              |
|10  |871              |
|11  |813              |
|12  |840              |
|13  |812              |
|14  |2150             |
|15  |810              |
|16  |813              |
|17  |850              |
|18  |819              |
|19  |858              |
+----+-----------------+
only showing top 20 rows

Temporal Patterns (Day of Week):
+-----------+-----------------+
|day_of_week|transaction_count|
+-----------+-----------------+
|6          |81

Data Storage Implementation

In [4]:
# Import necessary libraries
from pyspark.sql.functions import col, date_format, sum, count, avg, when, year, weekofyear
from pyspark.sql.types import TimestampType
from pymongo import MongoClient
from datetime import datetime, timedelta

# Ensure timestamp is in correct format
df = df.withColumn("timestamp_dt", col("timestamp").cast(TimestampType()))

# Add date column for partitioning
df = df.withColumn("date", date_format("timestamp_dt", "yyyy-MM-dd"))

# I. Load data into MongoDB with partitioning by date and merchant_id
try:
    df.write \
        .format("mongo") \
        .option("uri", "mongodb://localhost:27017/darooghe.transactions_partitioned") \
        .partitionBy("date", "merchant_id") \
        .mode("append") \
        .save()
    print("Partitioned transactions saved to MongoDB (transactions_partitioned)")
except Exception as e:
    print(f"Failed to save partitioned transactions: {e}")
    raise

# II. Retention policy: Keep last 24 hours of detailed data
try:
    # Filter recent transactions using Spark
    recent_transactions = df.filter(
        col("timestamp_dt") >= (datetime.now() - timedelta(hours=24))
    )
    
    # Write recent transactions to a new collection
    recent_transactions.write \
        .format("mongo") \
        .option("uri", "mongodb://localhost:27017/darooghe.transactions_recent") \
        .partitionBy("date", "merchant_id") \
        .mode("overwrite") \
        .save()
    print("Recent transactions (last 24 hours) saved to MongoDB (transactions_recent)")
    
    # Delete older data from transactions_partitioned using pymongo
    client = MongoClient('mongodb://localhost:27017/')
    db = client['darooghe']
    collection = db['transactions_partitioned']
    threshold = datetime.now() - timedelta(hours=24)
    result = collection.delete_many({"timestamp": {"$lt": threshold.isoformat()}})
    print(f"Deleted {result.deleted_count} transactions older than 24 hours")
except Exception as e:
    print(f"Failed to implement retention policy: {e}")
    raise

# III. Aggregated historical datasets
# Add customer segment, avoiding duplicate segment columns
try:
    # Drop existing segment column to prevent conflicts
    df = df.drop("segment")
    
    # Compute transaction frequency and segment
    customer_freq = df.groupBy("customer_id").agg(count("transaction_id").alias("transaction_count"))
    customer_freq = customer_freq.withColumn(
        "segment",
        when(col("transaction_count") > 20, "High Frequency")
        .when(col("transaction_count") > 5, "Medium Frequency")
        .otherwise("Low Frequency")
    ).alias("freq")
    
    # Join with alias to avoid ambiguity
    df = df.join(customer_freq, df.customer_id == customer_freq.customer_id, "left")
    
    # Summarized transaction data
    # Daily: By merchant_category and customer_segment
    daily_transaction_summary = df.groupBy("date", "merchant_category", "freq.segment") \
        .agg(
            sum("amount").alias("total_amount"),
            count("transaction_id").alias("transaction_count"),
            avg("amount").alias("avg_amount")
        )
    
    # Weekly: By year, week, and merchant_category
    weekly_transaction_summary = df.groupBy(
        year("timestamp_dt").alias("year"),
        weekofyear("timestamp_dt").alias("week"),
        "merchant_category"
    ) \
        .agg(
            sum("amount").alias("total_amount"),
            count("transaction_id").alias("transaction_count"),
            avg("amount").alias("avg_amount")
        )
    
    # Monthly: By month and customer_segment
    monthly_transaction_summary = df.groupBy(
        date_format("timestamp_dt", "yyyy-MM").alias("month"),
        "freq.segment"
    ) \
        .agg(
            sum("amount").alias("total_amount"),
            count("transaction_id").alias("transaction_count"),
            avg("amount").alias("avg_amount")
        )
    
    # Save transaction summaries
    daily_transaction_summary.write \
        .format("mongo") \
        .option("uri", "mongodb://localhost:27017/darooghe.daily_transaction_summary") \
        .mode("overwrite") \
        .save()
    weekly_transaction_summary.write \
        .format("mongo") \
        .option("uri", "mongodb://localhost:27017/darooghe.weekly_transaction_summary") \
        .mode("overwrite") \
        .save()
    monthly_transaction_summary.write \
        .format("mongo") \
        .option("uri", "mongodb://localhost:27017/darooghe.monthly_transaction_summary") \
        .mode("overwrite") \
        .save()
    print("Transaction summaries (daily, weekly, monthly) saved to MongoDB")
except Exception as e:
    print(f"Failed to save transaction summaries: {e}")
    raise

# Commission reports
try:
    # Daily: By merchant_category
    commission_reports_daily = df.groupBy("date", "merchant_category") \
        .agg(
            sum("commission_amount").alias("total_commissions"),
            count("transaction_id").alias("transaction_count")
        )
    
    # Monthly: By merchant_category
    commission_reports_monthly = df.groupBy(
        date_format("timestamp_dt", "yyyy-MM").alias("month"),
        "merchant_category"
    ) \
        .agg(
            sum("commission_amount").alias("total_commissions"),
            count("transaction_id").alias("transaction_count")
        )
    
    # Save commission reports
    commission_reports_daily.write \
        .format("mongo") \
        .option("uri", "mongodb://localhost:27017/darooghe.commission_reports_daily") \
        .mode("overwrite") \
        .save()
    commission_reports_monthly.write \
        .format("mongo") \
        .option("uri", "mongodb://localhost:27017/darooghe.commission_reports_monthly") \
        .mode("overwrite") \
        .save()
    print("Commission reports (daily, monthly) saved to MongoDB")
except Exception as e:
    print(f"Failed to save commission reports: {e}")
    raise

# Display results for verification
print("Daily Transaction Summary:")
daily_transaction_summary.show(5, truncate=False)
print("Weekly Transaction Summary:")
weekly_transaction_summary.show(5, truncate=False)
print("Monthly Transaction Summary:")
monthly_transaction_summary.show(5, truncate=False)
print("Daily Commission Reports:")
commission_reports_daily.show(5, truncate=False)
print("Monthly Commission Reports:")
commission_reports_monthly.show(5, truncate=False)

Partitioned transactions saved to MongoDB (transactions_partitioned)
Recent transactions (last 24 hours) saved to MongoDB (transactions_recent)
Deleted 3001 transactions older than 24 hours
Transaction summaries (daily, weekly, monthly) saved to MongoDB
Commission reports (daily, monthly) saved to MongoDB
Daily Transaction Summary:
+----------+-----------------+----------------+------------+-----------------+------------------+
|date      |merchant_category|segment         |total_amount|transaction_count|avg_amount        |
+----------+-----------------+----------------+------------+-----------------+------------------+
|2025-04-26|entertainment    |Medium Frequency|893620577   |896              |997344.3939732143 |
|2025-04-25|entertainment    |High Frequency  |1137588090  |1123             |1012990.284951024 |
|2025-04-25|transportation   |Medium Frequency|574757774   |554              |1037468.9061371841|
|2025-04-25|food_service     |Medium Frequency|563822812   |553              |